In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast
import spark_util

In [2]:
submit_args = "--conf spark.hadoop.fs.s3a.endpoint=http://minio-ml-workshop:9000 \
--conf spark.hadoop.fs.s3a.access.key=minio \
--conf spark.hadoop.fs.s3a.secret.key=minio123 \
--conf spark.hadoop.fs.s3a.path.style.access=true \
--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem \
--conf spark.hadoop.fs.s3a.multipart.size=104857600 \
--packages org.apache.hadoop:hadoop-aws:3.2.0"

spark = spark_util.getOrCreateSparkSession("Clean flights data", submit_args)
spark.sparkContext.setLogLevel("INFO")

Initializing environment variables for Spark
Creating a spark session...
Spark session created


In [3]:
data_location = "s3a://flights-data/flights/*.parquet"
df_flights = spark.read.parquet(data_location)
#df_flights.printSchema()
#df_flights.count()

In [29]:
from pyspark.sql.functions import col, approx_count_distinct

df_distinct = df_flights.agg(*(approx_count_distinct(col(c)).alias(c) for c in df_flights.columns))

first = df_distinct.first()
for col in df_distinct.columns:
    print(col)



[<generator object <genexpr> at 0x7f31cdb7f270>]

In [30]:
df_clean = df_flights.drop("year", "ORIG_COUNTRY", "DEST_COUNTRY", "ORIG_IATA_CODE", "DEST_IATA_CODE")
df_clean.printSchema()

root
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- airline: string (nullable = true)
 |-- flight_number: integer (nullable = true)
 |-- tail_number: string (nullable = true)
 |-- origin_airport: string (nullable = true)
 |-- destination_airport: string (nullable = true)
 |-- scheduled_departure: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- departure_delay: integer (nullable = true)
 |-- taxi_out: integer (nullable = true)
 |-- wheels_off: string (nullable = true)
 |-- scheduled_time: integer (nullable = true)
 |-- elapsed_time: integer (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- wheels_on: string (nullable = true)
 |-- taxi_in: integer (nullable = true)
 |-- scheduled_arrival: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- arrival_delay: integer (nullable = true)
 |-- diverted: integer (nullab

In [ ]:
spark.stop()